
<h3 style="color:#3295ec;"><center>تمرین دوم درس شبکه های عصبی مصنوعی - قسمت اول</center></h3>
<p style="text-align: right;">نام و نام خانوادگی :فرامرز نادری</p>
<p style="text-align: right;">97722203:شماره دانشجویی</p>

In [2]:
#data
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data as mnist_data
mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=True, validation_size=0)
print('Test shape:',mnist.test.images.shape)
print('Train shape:',mnist.train.images.shape)

Extracting data\train-images-idx3-ubyte.gz
Extracting data\train-labels-idx1-ubyte.gz
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz
Test shape: (10000, 784)
Train shape: (60000, 784)


In [3]:
# TensorFlow Graph visualizer code     https://www.youtube.com/watch?v=FrEp94A9FoY
import numpy as np
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [4]:
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("float", [None, 10])

<p style="text-align: right;">با یک لایه مخفی MLP :ساختار پیشنهادی</p>
<p style="text-align: right;">learning_rate ,batch_size ,num_epochs ,num_hidden_layer ,num_input ,num_output ,function:پارامترهای شبکه</p>
<p style="text-align: right;">علت پیشنهاد این ساختار:طبق جستجو هایی که کردم هیچ فرمول مشخصی برای تعیین تعداد لایه های مخفی و تعداد نورون های آنها نبود من هم براساس نتایج پیاده سازی های قبلی و آزمون و خطا این ساختار را انتخاب کردم   </p>


In [5]:
# model
#hidden layer
w1= tf.Variable(tf.random_normal([784, 800]))
b1= tf.Variable(tf.random_normal([800]))
layer_1 = tf.add(tf.matmul(x, w1), b1) 
layer_1 = tf.nn.relu(layer_1)
#output
wout= tf.Variable(tf.random_normal([800, 10]))   
biasout= tf.Variable(tf.random_normal([10]))
pred = tf.matmul(layer_1, wout) + biasout  


In [6]:
# Calculate loss
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits= pred))

# Optimize loss
optimizer = tf.train.AdamOptimizer(learning_rate=0.003).minimize(cross_entropy)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# Confusion matrix
confusion_matrix_tf = tf.confusion_matrix(tf.argmax(y, 1), tf.argmax(pred, 1))

In [9]:
# Launch the graph
with tf.Session() as sess:
    # Initializing the variables
    init = tf.global_variables_initializer()
    sess.run(init)
    num_epochs=10
    batch_size=100
    for epoch in range(num_epochs):
            E_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                batch_x,  batch_label = mnist.train.next_batch(batch_size)
                _, l = sess.run([optimizer, cross_entropy], feed_dict={x: batch_x, y:  batch_label})
                E_loss = E_loss + l

            print('Epoch', epoch, 'loss:',E_loss)

    print ("Accuracy:", accuracy.eval(feed_dict={x: mnist.train.images, y: mnist.train.labels}))
    print ("Accuracy:", accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))
    cm=confusion_matrix_tf.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels})

Epoch 0 loss: 7387.306629002094
Epoch 1 loss: 2068.875917841602
Epoch 2 loss: 1182.689523109253
Epoch 3 loss: 753.7162845146484
Epoch 4 loss: 528.9654110911023
Epoch 5 loss: 402.39020570429005
Epoch 6 loss: 368.9394027647644
Epoch 7 loss: 295.95439672657454
Epoch 8 loss: 236.69592155234423
Epoch 9 loss: 229.34413871487268
Accuracy: 0.9892833
Accuracy: 0.9626


In [10]:
print(cm)

[[ 968    0    0    0    1    1    6    2    2    0]
 [   0 1126    3    0    0    1    2    2    0    1]
 [   7    4  990    4    1    0    3   14    7    2]
 [   2    1    7  945    0   35    0   10    2    8]
 [   3    4    5    0  929    1    5    6    4   25]
 [   5    0    0    4    0  871    7    1    1    3]
 [   6    1    0    1    2    6  940    1    1    0]
 [   0    0    9    0    2    0    0 1010    2    5]
 [  12    1    7   15    2   14    8    9  898    8]
 [  13    4    1    3    5    9    1   21    3  949]]


In [11]:
gr = tf.Graph()

with gr.as_default():
    # model
    with tf.name_scope('Input'):
        x = tf.placeholder("float", [None, 784],name='x')
    with tf.name_scope('HiddenLayer'):
        w1= tf.Variable(tf.random_normal([784, 800]),name='w')
        b1= tf.Variable(tf.random_normal([800]),name='b')
        layer_1 = tf.add(tf.matmul(x, w1,name='XW_for_layer1'), b1,name='XWb_for_layer1') 
        layer_1 = tf.nn.relu(layer_1,name='relu_func_for_layer1')
    with tf.name_scope('Output'):
        wout= tf.Variable(tf.random_normal([800, 10]),name='w_output')   
        biasout= tf.Variable(tf.random_normal([10]),name='b_output')
        pred = tf.matmul(layer_1, wout,name='output_layer1W') + biasout  
    
tf.summary.FileWriter("logs", gr).close()

In [12]:
show_graph(gr)

<img src="plot.png" >
<img src="plot2.png" >